# RNN から Attention までのモデルを定義できるようになる

## 深層学習での前処理の順序

分かち書き・正規化
- 単語毎に分割(分かち書き)・正規化(無駄な文字を削除)

ここまでをクレンジング(cleansing)という。

↓

単語埋め込み(Word Embedding)
- 単語をベクトル化

または、

単語をトークン化(Tokenize)
- 単語を一意の ID に変換

↓

パディング(Padding)・端数処理(truncating)

↓

学習モデルに入力

## Word Embedding(単語埋め込み)

古典的なNLPでは単語を数値に変換して計算を行う。

- One hot vector
- TFIDF

しかし、これらの手法は、「**スパース**(まばらで単語の本質的な特徴が少ない)」なので、効率が悪い。
一方、ニューラルネットワークでは、「**Embedding**(各単語にわずか数百次元のベクトルを割り当てる)」を行うことで、「**Word2Vec**」 のように似た意味を持った単語ベクトルを得られる。

Word Embedding のことを、「**分散表現**」とも言う。

---

`tensorflow.keras`では、`Embedding Layer`で Embedding を行う。

Embedding への入力は、各単語に割り当てた ID からなる行列(**Embedding Matrix**)となる。

これら３つで、最低限指定できる。

- input_dim: 語彙数、単語の種類の数
- output_dim: 単語ベクトルの次元
- input_length: 各文の長さ

以下を追加↓

`from tensorflow.keras.layers import Embedding`

`vocab_size = 1000 # 扱う語彙の数`

`embedding_dim = 100 # 単語ベクトルの次元`

`seq_length = 20 # 文の長さ`



また、頻度が一定以下の単語は学習にあまり貢献しないので、 Unknown(未知語)にすると、メモリを節約できる。`<unk>`で表現する。

おもしろページ↓

[単語埋め込み (Word embeddings):Tensorflow](https://www.tensorflow.org/tutorials/text/word_embeddings?hl=ja)


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential

batch_size = 32 # バッチサイズ(一度に並列計算を行うデータ数)

input_data = np.arange(batch_size * seq_length).reshape(batch_size, seq_length)

model = Sequential()
# Embeddingを追加 -> keras embedding で検索

output = model.predict(input_data)

# Embedding Matrix の要素を得られる
print(output.shape)

## 活性化関数(Activation function)

ニューラルネットワークでは、非線形関数で出力することで複雑なモデル(非線形モデル)を近似(**学習**)できる

入力 $=$ 入力 $\times$ 重み $+$ バイアス

出力 $=$ **活性化関数** $($ 複数のニューロンからの入力 $\times$ 各重み $+$ バイアス $)$

[ニューラルネットワークの基礎](https://tutorials.chainer.org/ja/13_Basics_of_Neural_Networks.html)

---

tensorflow.keras.models.Sequentialを使わずにこのようにモデルを記述する方法を`Functional API`という。

`y = Activation('softmax')(x)`

In [ ]:
import numpy as np
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model


x = Input(shape=(20, 5))
# xにsoftmaxを作用させる


model = Model(inputs=x, outputs=y)

sample_input = np.ones((12, 20, 5))
sample_output = model.predict(sample_input)

print(np.sum(sample_output, axis=2))

## RNN(Recurrent Neural Network, 再帰ニューラルネットワーク)

「**可変長系列**(任意の長さの入力)」の時系列データの分析に使える。

改良された亜種が多く、Simple RNN が使われることは少ないが、

`from tensorflow.keras.layers import SimpleRNN` でモジュールをインポートし、

`model.add(SimpleRNN(ユニット数, return_sequences=True))`

でモデルを追加できる。

---

- 時間：$t$
- 活性化関数：$f$
- 隠れ状態ベクトル(隠れ層のベクトル)：$h_t$
- 時刻$t$の入力：$y_t$
- 時刻$t$の出力：$y_t$
- 学習可能な行列：$W_h, W_y, U$
- バイアス：$b_h, b_y$

$\color{red}{Uh_{t-1}}$ で隠れ層の時間を戻している。$h_{t-1}$でデータをループさせ、時系列を分析できる。

$\begin{align*}
h_t &= f^{\rm{(hidden})}(W_{h}x_t + \color{red}{Uh_{t-1}} + b_{h})\\
y_{t} &= f^{(\rm{out})}(W_{y}h_t + b_{y}) 
\end{align*}$

分かりやすい図↓

https://axa.biopapyrus.jp/deep-learning/rnn/

1990年、エルマンネット(RNNの原形)発表。

https://www.cis.twcu.ac.jp/~asakawa/waseda2002/elman.pdf


## 過学習(Over fitting)防止の戦略

ニューラルネットワークでは、学習を行い過ぎることで訓練データ(Train Data)では、十分な精度を出せるが、未知のデータに対して全く精度を出せない場合がある。

### 汎化性能

未知のデータでも高い性能を出せる性能を「**汎化性能**」という。

ニューラルネットワークの学習では、学習しすぎ(**過学習**)でも、学習不足でもいけない。


### 局所最適解と大域最適解(全体最適解)

過学習は、与えられたデータ内だけで、局所最適解に陥っている状態。

ニューラルネットワークでは、大域最適解に近づけることが目的。

[全体最適解と局所最適解](https://qiita.com/9ryuuuuu/items/ab361d00af32f71c1f89)


### Dropout(ドロップアウト)

訓練時、ランダム(今回、`0.3`)にニューロン(ユニット)を`0`(不活性化)にして、過学習を防ぐ手法。

---

`y = Dropout(0.3)(x)`


In [ ]:
import numpy as np
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.models import Model

batch_size = 32  # バッチサイズ
vocab_size = 1000  # 扱う語彙の数
embedding_dim = 100  # 単語ベクトルの次元
seq_length = 20  # 文1の長さ
lstm_units = 200  # LSTMの隠れ状態ベクトルの次元数

input = Input(shape=(seq_length,))

embed = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  input_length=seq_length)(input)

rnn = SimpleRNN(lstm_units, return_sequences=True)(embed)

output = 

model = Model(inputs=input, outputs=output)

sample_input = np.arange(
    batch_size * seq_length).reshape(batch_size, seq_length)

sample_output = model.predict(sample_input)

print(sample_output.shape)

### 正則化

L1正規化
 - 重み係数の絶対値に比例するコストを加える

L2正則化
 - 重み係数の二乗に比例するコストを加える。

がある。

レイヤーの重みに制約を付けることでパラメータを単純にすることで、モデルも単純になる。


## LSTM(Long Short Term Memory, 長短期記憶)

RNNの改良版で、RNNに時間情報を出し入れする複数のゲートを設けることで長期記憶と短期記憶を実現している。

1997年発表。

https://www.bioinf.jku.at/publications/older/2604.pdf

---

以下を追加↓

`from tensorflow.keras.layers import LSTM`

`lstm_units = 200` # LSTMの隠れ状態ベクトルの次元数(100~300程度が目安)

そして、モデルに`LSTM`を追加する。

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
# モジュールをインポート



batch_size = 32 # バッチサイズ(一度に並列計算を行うデータ数)
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length = 20 # 文の長さ

input_data = np.arange(batch_size * seq_length).reshape(batch_size, seq_length)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length))
# LSTMを追加 -> keras lstm で検索



output = model.predict(input_data)
print(output.shape)

## BiLSTM(Bi-Directional Recurrent Neural Network, 双方向性再帰型ニューラルネットワーク)

従来のLSTMと同じ過去の情報に加え、予測した未来の情報も加える。過去から未来までのすべての入力系列を加味して分析することで、高い精度を得られる手法。

未来への順伝播 $x={x1, ... ,xT}$ 

と

過去への逆伝播 $x={xT, ... , x1}$ の双方向に伝播するネットワーク。

図↓

https://axa.biopapyrus.jp/deep-learning/rnn/brnn.html

1997年。Bidirectional recurrent neural net-works. として、発表。

---

モジュールをインポート

`from tensorflow.keras.layers import Bidirectional`


`tensorflow.keras`では、双方向の入力系列を繋げるためのオプションは複数ある。

- sum: 要素和
- mul: 要素積
- concat: 結合
- ave: 平均
- None: 結合せずにlistを返す



In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding


batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length = 20 # 文の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数

input_data = np.arange(batch_size * seq_length).reshape(batch_size, seq_length)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length))

# LSTMをの入力系列を要素和させる -> keras bidirectional で検索


# 次元数が減っている事が分かる
output = model.predict(input_data)
print(output.shape)

## Attention

深層学習における初の実用的な機械翻訳モデル。

$t$の各時刻において$s$の各時刻の隠れ状態ベクトルを考慮した特徴を計算する。

加重平均

**双方向でも適用可能**なので**BERT**や**GPT-2**といったAttentionを応用したモデルが台頭する。

[Neural Machine Translation by Jointly Learning to Align and Translate(Bahdanau 2015)](https://arxiv.org/abs/1409.0473)で発表。

[Attention Is All You Need(Vaswani et al, 2017)](https://arxiv.org/abs/1706.03762) で Attention で構成された **Transformer** が開発され、汎用的に高い精度を叩き出し、有名になった。

---

`dot([u, v], axes=2)`は、`u`と`v`のバッチごとの行列積を計算します。



In [ ]:
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import dot, concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model

batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length1 = 20 # 文1の長さ
seq_length2 = 30 # 文2の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数
hidden_dim = 200 # 最終出力のベクトルの次元数

# 2つのLSTMに共通のEmbeddingLayerを使うため、はじめにEmbeddingLayerを定義します。
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

input1 = Input(shape=(seq_length1,))
embed1 = embedding(input1)
bilstm1 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed1)

input2 = Input(shape=(seq_length2,))
embed2 = embedding(input2)
bilstm2 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed2)

# 要素ごとの積を計算する
product = dot([bilstm2, bilstm1], axes=2) # サイズ：[バッチサイズ、文2の長さ、文1の長さ]

# ここにAttention mechanismを実装してください
a = 
c = 
c_bilstm2 = 
h = 

model = Model(inputs=[, ], outputs=)

sample_input1 = np.arange(batch_size * seq_length1).reshape(batch_size, seq_length1)
sample_input2 = np.arange(batch_size * seq_length2).reshape(batch_size, seq_length2)

sample_output = model.predict([sample_input1, sample_input2])
print(sample_output.shape)